# Getting Started with Prompt Engineering

This notebook contains examples and exercises to learning about prompt engineering.
It is adapted to the Google AI Studio API from original OpenAI used in the [prompt engineering guide by DAIR.AI | Elvis Saravia](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-lecture.ipynb)


---

## 0. Create Google AI Studio account and get API keys

1. Go to https://aistudio.google.com/
2. Sign in with your Google account
3. Go to https://ai.google.dev/gemini-api/docs/api-key and get the API key

If you want, check out https://ai.google.dev/gemini-api/docs/quickstart for more details.

## 1. Prompt Engineering Basics

Objectives
- Load the libraries
- Review the format
- Cover basic prompts
- Review common use cases

Below we are loading the necessary libraries, utilities, and configurations.

In [ ]:
%%capture
# update or install the necessary libraries
!pip install --upgrade google-genai
!pip install --upgrade python-dotenv
#!pip install --upgrade langchain
#!pip install langchain-community langchain-core


In [37]:

from google import genai
from google.genai import types

import os
import IPython
from dotenv import load_dotenv

Load environment variables. You can use anything you like but I used `python-dotenv`. Just create a `.env` file with your `GEMINI_API_KEY` then load it.

In [10]:
load_dotenv()

True

Let's check that the API works as expected.

In [19]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
response = client.models.generate_content(
    model="gemini-2.0-flash", contents="Explain how AI works"
)
print(response.text)


Okay, let's break down how AI (Artificial Intelligence) works.  It's a broad field, so I'll give you a general overview and then touch on some key techniques.

**Core Idea: Mimicking Human Intelligence**

At its heart, AI aims to create machines that can perform tasks that typically require human intelligence. This includes:

*   **Learning:** Acquiring information and rules for using the information.
*   **Reasoning:**  Using information to draw conclusions and make decisions.
*   **Problem-solving:**  Finding solutions to complex problems.
*   **Perception:** Understanding sensory input (like images, sounds, or text).
*   **Natural Language Processing (NLP):** Understanding and generating human language.

**How AI Achieves This (Key Concepts)**

AI relies on algorithms and data.  Algorithms are sets of instructions that tell a computer how to perform a specific task. Here's a breakdown of some fundamental concepts:

1.  **Data:**  AI systems learn from data. The more data, the better

In [38]:
def set_params(
    model="gemini-2.0-flash",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set Gemini parameters"""

    gemini_params = {}    

    gemini_params['model'] = model
    gemini_params['temperature'] = temperature
    gemini_params['max_output_tokens'] = max_tokens
    gemini_params['top_p'] = top_p
    gemini_params['frequency_penalty'] = frequency_penalty
    gemini_params['presence_penalty'] = presence_penalty
    return gemini_params

def get_completion(params, prompt):
    """ GET completion from Google Gemini api"""

    response = client.models.generate_content(
        model = params['model'],
        contents=prompt,
        config=types.GenerateContentConfig(
            temperature=params['temperature'],
            top_p=params['top_p'],
            max_output_tokens=params['max_output_tokens'],
            presence_penalty=params['presence_penalty'],
            frequency_penalty=params['frequency_penalty'],
        ))
    return response


Basic prompt example:

In [39]:
# basic example
params = set_params()

prompt = "The sky is"

response = get_completion(params, prompt)

In [40]:
response.text

"The sky is **blue** (on a typical sunny day).\n\nHowever, it could also be:\n\n*   **Gray** (when it's cloudy)\n*   **Dark** (at night)\n*   **Orange, pink, or purple** (during sunrise or sunset)\n*   **Black** (in space)\n\nIs there any specific time or condition you'd like me to describe the sky in?"

Try with different temperature to compare results:

In [47]:
params = set_params(temperature=0)
response = get_completion(params, prompt)
IPython.display.Markdown(response.text)

The sky is **blue** (on a typical sunny day).

However, it could also be:

*   **Gray** (on a cloudy day)
*   **Orange/Pink/Red** (during sunrise or sunset)
*   **Black** (at night)
*   **Purple** (sometimes during twilight or storms)
*   **Full of stars** (at night, away from city lights)
*   **Overcast**
*   **Clear**

What would you like me to elaborate on?


### 1.1 Text Summarization

In [48]:
params = set_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance. 

Explain the above in one sentence:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.text)

Antibiotics are medications that combat bacterial infections by killing or inhibiting bacterial growth, taken orally or intravenously, but are ineffective against viruses and can contribute to antibiotic resistance if misused.


Exercise: Instruct the model to explain the paragraph in one sentence like "I am 5". Do you see any differences?

### 1.2 Question Answering

In [49]:
prompt = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: What was OKT3 originally sourced from?

Answer:"""


response = get_completion(params, prompt)
IPython.display.Markdown(response.text)


Mice


Context obtained from here: https://www.nature.com/articles/d41586-023-00400-x

Exercise: Edit prompt and get the model to respond that it isn't sure about the answer. 

### 1.3 Text Classification

In [50]:
prompt = """Classify the text into neutral, negative or positive.

Text: I think the food was okay.

Sentiment:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.text)

Sentiment: **Neutral**


Exercise: Modify the prompt to instruct the model to provide an explanation to the answer selected. 

### 1.4 Role Playing

In [51]:
prompt = """The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""


response = get_completion(params, prompt)
IPython.display.Markdown(response.text)

Affirmative. Black holes, celestial objects exhibiting gravitational fields so intense that no matter or radiation can escape, primarily form through two well-established astrophysical mechanisms:

1.  **Stellar Collapse:** This is the most common pathway. When a massive star (typically exceeding 10-20 times the mass of our Sun) exhausts its nuclear fuel, it can no longer sustain the outward pressure required to counteract the inward pull of gravity. The star's core rapidly collapses under its own weight. If the core's mass exceeds the Tolman-Oppenheimer-Volkoff (TOV) limit (approximately 2.1 solar masses, though this value is subject to ongoing refinement based on the equation of state of nuclear matter), the collapse becomes unstoppable, leading to the formation of a stellar-mass black hole. The outer layers of the star are often ejected in a supernova explosion.

2.  **Direct Collapse:** In certain less common scenarios, a very massive protostar (potentially hundreds or thousands of solar masses) can bypass the standard nuclear fusion processes due to extremely rapid accretion of material. Without the outward radiation pressure generated by nuclear reactions, the protostar undergoes a direct collapse into a black hole without a supernova. This process is hypothesized to

Exercise: Modify the prompt to instruct the model to keep AI responses concise and short.

### 1.5 Code Generation

In [52]:
prompt = "\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n\"\"\""

response = get_completion(params, prompt)
IPython.display.Markdown(response.text)


```sql
SELECT s.StudentName
FROM students s
JOIN departments d ON s.DepartmentId = d.DepartmentId
WHERE d.DepartmentName = 'Computer Science';
```

### 1.6 Reasoning

In [53]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""


response = get_completion(params, prompt)
IPython.display.Markdown(response.text)

Okay, let's break this down:

1.  **Identify the odd numbers:** From the list 15, 32, 5, 13, 82, 7, 1, the odd numbers are 15, 5, 13, 7, and 1.

2.  **Add the odd numbers:** 15 + 5 + 13 + 7 + 1 = 41

3.  **Determine if the result is odd or even:** 41 is an odd number.

**Therefore, the sum of the odd numbers in the group is 41, which is an odd number.**


Exercise: Improve the prompt to have a better structure and output format.

## 2. Advanced Prompting Techniques

Objectives:

- Cover more advanced techniques for prompting: few-shot, chain-of-thoughts,...

### 2.2 Few-shot prompts

In [54]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""


response = get_completion(params, prompt)
IPython.display.Markdown(response.text)

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1.
Odd numbers: 15, 5, 13, 7, 1
Sum: 15 + 5 + 13 + 7 + 1 = 41
41 is an odd number.
Therefore, the statement is False.

A: The answer is False.


### 2.3 Chain-of-Thought (CoT) Prompting

In [55]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.text)

Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.


### 2.4 Zero-shot CoT

In [57]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""


response = get_completion(params, prompt)
IPython.display.Markdown(response.text)

1. **Start:** You began with 10 apples.
2. **Gave to neighbor:** You gave away 2 apples, so you had 10 - 2 = 8 apples.
3. **Gave to repairman:** You gave away another 2 apples, so you had 8 - 2 = 6 apples.
4. **Bought more:** You bought 5 more apples, so you had 6 + 5 = 11 apples.
5. **Ate one:** You ate 1 apple, so you had 11 - 1 = 10 apples.

**Answer:** You remained with 10 apples.


### 2.5 Self-Consistency
As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#self-consistency) and try them here. 

### 2.6 Generate Knowledge Prompting

As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#generated-knowledge-prompting) and try them here. 

---